## 모델 동작에 필요한 사전 준비

In [ ]:
! pip install tensorflow_datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import time
import matplotlib.pyplot as plt

from IPython.display import clear_output

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

## 데이터 불러오기 및 전처리

In [ ]:
# 이번 CycleGAN 실습에 활용될 말과 얼룩말 이미지 데이터를 불러온 후,
# 불러온 데이터를 Training용과 Testing용으로 나누는 과정
# 모든 이미지 데이터의 크기는 256 x 256 x 3 (RGB)

dataset, metadata = tfds.load('cycle_gan/horse2zebra', with_info = True, as_supervised = True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

In [ ]:
# Hyperparameter 값들

BUFFER_SIZE = 1000  # Dataset의 순서를 섞는 과정에서의 Buffer의 크기
BATCH_SIZE = 1      # Training 및 Test 과정에서의 (Mini)Batch의 크기
IMG_HEIGHT = 256    # 조정될 그림의 높이
IMG_WIDTH = 256     # 조정될 그림의 폭

### 이미지 데이터 전처리용 함수

In [ ]:
# IMG_HEIGHT x IMG_WIDTH (256 x 256) 크기보다 큰 이미지의 경우
# 이미지 내에서 무작위로 IMG_HEIGHT x IMG_WIDTH 크기만큼의 부분 추출

def random_crop(image):
    cropped_image = tf.image.random_crop(image, size = [IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image



# Data augmentation - 데이터셋의 이미지를 무작위적으로 변환
# 여기서는 이미지의 일부를 무작위로 추출한 후 무작위로 추가적인 좌우 반전 과정을 거침

def random_jitter(image):
    # 256 x 256 x 3 크기의 이미지를 286 x 286 x 3 크기로 조정
    image = tf.image.resize(image, [286, 286], method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # 조정된 이미지에서 256 x 256 크기 부분 추출 -> 이미지의 일부가 무작위로 추출 및 확대되는 효과
    image = random_crop(image)

    # 추출해서 얻어진 이미지를 그대로 두거나 좌우 반전
    image = tf.image.random_flip_left_right(image)

    return image



# 이미지 내의 각 픽셀 내의 값을 [-1, 1] 범위로 표준화

def normalize(image):
    image = tf.cast(image, tf.float32)  # 표준화 전: 0 ~ 255 (RGB)
    image = (image / 127.5) - 1         # 표준화 후: -1 ~ 1 (RGB)
    return image

### Training 및 Test 이미지 데이터 전처리 함수

In [ ]:
# Training 이미지 데이터만 이미지 변형 (data augmentation) 과정을 거침

def preprocess_image_train(image, label):
    image = random_jitter(image)
    image = normalize(image)
    return image



def preprocess_image_test(image, label):
    image = normalize(image)
    return image

In [ ]:
# 말과 얼룩말 이미지 데이터 전처리
# 전처리 과정에서 이미지의 순서 또한 무작위적으로 섞이게 됨!

train_horses = train_horses.map(preprocess_image_train, num_parallel_calls = AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(1)
train_zebras = train_zebras.map(preprocess_image_train, num_parallel_calls = AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(1)

test_horses = test_horses.map(preprocess_image_test, num_parallel_calls = AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(1)
test_zebras = test_zebras.map(preprocess_image_test, num_parallel_calls = AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(1)

In [ ]:
# 이미지 예시 확인을 위해 전처리된 Training용 이미지들 중 가장 첫번째 이미지를 불러옴

sample_horse = next(iter(train_horses))
sample_zebra = next(iter(train_zebras))

In [ ]:
# 말 이미지 예시

plt.subplot(121)
plt.title('Horse\n(Original Image)')
plt.imshow(sample_horse[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Horse with Random Jitter\n(Augmented Image)')
plt.imshow(random_jitter(sample_horse[0]) * 0.5 + 0.5)

In [ ]:
# 얼룩말 이미지 예시

plt.subplot(121)
plt.title('Zebra\n(Original Image)')
plt.imshow(sample_zebra[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Zebra with Random Jitter\n(Augmented Image)')
plt.imshow(random_jitter(sample_zebra[0]) * 0.5 + 0.5)

## GAN의 구성 요소 구현

### 이미지와 벡터 사이의 변환 - Downsampling과 Upsampling

In [ ]:
# Downsampling: 이미지 -> 벡터
# 매 Downsampling 과장마다 이미지 크기 (가로, 세로 길이)가 절반으로 축소됨

def downsample(filters, size, apply_batchnorm = True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    
    # 이미지 크기를 절반으로 축소시키는 과정
    # 2D convolutional layer에서 same padding을 적용하고 strides 값을 2로 할 경우 이미지의 크기가 절반으로 줄어드는 효과!
    result.add(tf.keras.layers.Conv2D(filters, size, strides = 2, padding = 'same', kernel_initializer = initializer, use_bias = False))
    
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result



# Upsampling: 벡터 -> 이미지
# 매 Upsampling 과정마다 이미지 크기 (가로, 세로 길이) 두 배로 확장됨

def upsample(filters, size, apply_dropout = False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    
    # 이미지 크기를 두 배로 확장시키는 과정
    # 2D convolutional layer의 역연산에 해당하는 transposed 2D convolutional layer의 경우,
    # same padding을 적용하고 strides 값을 2로 할 경우 이미지의 크기가 두 배로 확장되는 효과!
    result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides = 2, padding = 'same', kernel_initializer = initializer, use_bias = False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

### GAN의 생성자와 판별자 정의

In [ ]:
# GAN의 생성자 (Generator)

def Generator():
    inputs = tf.keras.layers.Input(shape = [256, 256, 3])

    down_stack = [
        downsample(64, 4, apply_batchnorm = False), # (batch_size, 128, 128, 64)
        downsample(128, 4),                         # (batch_size, 64, 64, 128)
        downsample(256, 4),                         # (batch_size, 32, 32, 256)
        downsample(512, 4),                         # (batch_size, 16, 16, 512)
        downsample(512, 4),                         # (batch_size, 8, 8, 512)
        downsample(512, 4),                         # (batch_size, 4, 4, 512)
        downsample(512, 4),                         # (batch_size, 2, 2, 512)
        downsample(512, 4),                         # (batch_size, 1, 1, 512)
    ]


    # Upsampling의 경우 channel 수가 upsample 함수의 parameter로 제공되는 channel 수의 2배!
    # 이는 upsampling 과정에서는 skip connection을 추가적으로 진행하기 때문 (아래 for문들 참고)
    up_stack = [
        upsample(512, 4, apply_dropout = True),     # (batch_size, 2, 2, 1024)
        upsample(512, 4, apply_dropout = True),     # (batch_size, 4, 4, 1024)
        upsample(512, 4, apply_dropout = True),     # (batch_size, 8, 8, 1024)
        upsample(512, 4),                           # (batch_size, 16, 16, 1024)
        upsample(256, 4),                           # (batch_size, 32, 32, 512)
        upsample(128, 4),                           # (batch_size, 64, 64, 256)
        upsample(64, 4),                            # (batch_size, 128, 128, 128)
    ]


    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(3, 4, strides = 2, padding = 'same', kernel_initializer = initializer, activation = 'tanh')      # (batch_size, 256, 256, 3)

    x = inputs

    # 모델 내에서의 Downsampling 과정
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])
    

    # 모델 내에서의 Upsampling 과정
    # 이전 정보의 보존을 목적으로 skip connection도 추가
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])
    
    x = last(x)
    

    return tf.keras.Model(inputs = inputs, outputs = x)

In [ ]:
# GAN의 판별자 (Discriminator)

def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    
    inputs = tf.keras.layers.Input(shape = [256, 256, 3], name = 'input_image')
    
    down1 = downsample(64, 4, False)(inputs)    # (batch_size, 128, 128, 64)
    down2 = downsample(128, 4)(down1)           # (batch_size, 64, 64, 128)
    down3 = downsample(256, 4)(down2)           # (batch_size, 32, 32, 256)
    
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)      # (batch_size, 34, 34, 64)
    conv = tf.keras.layers.Conv2D(128, 4, strides = 1, kernel_initializer = initializer, use_bias = False)(zero_pad1)     # (batch_size, 31, 31, 64)
    
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
    
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)     # (batch_size, 33, 33, 64)
    last = tf.keras.layers.Conv2D(1, 4, strides = 1, kernel_initializer = initializer)(zero_pad2)       # (batch_size, 30, 30, 1)
    
    return tf.keras.Model(inputs = inputs, outputs = last)

### CycleGAN 구조 정의

In [ ]:
# CycleGAN의 경우 2개의 생성자와 2개의 판별자로 구성됨

''' Fill here '''

### 생성자 동작 확인

In [ ]:
# generator_g: 말 -> 얼룩말
# generator_f: 얼룩말 -> 말

to_zebra = generator_g(sample_horse)
to_horse = generator_f(sample_zebra)

plt.figure(figsize = (8, 8))
contrast = 8

imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

for i in range(len(imgs)):
    plt.subplot(2, 2, i + 1)
    plt.title(title[i])
    if i % 2 == 0:
        plt.imshow(imgs[i][0] * 0.5 + 0.5)
    else:
        plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
        
plt.show()

### 판별자 동작 확인

In [ ]:
# discriminator_x: 제공받은 이미지가 말 이미지인지 판별
# discriminator_y: 제공받은 이미지가 얼룩말 이미지인지 판별

plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is This a Real Zebra?')
plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap = 'RdBu_r')

plt.subplot(122)
plt.title('Is This a Real Horse?')
plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap = 'RdBu_r')

plt.show()

## Training

In [ ]:
# Training 과정 관련 hyperparameters

LAMBDA = 10
EPOCHS = 1

In [ ]:
# Loss function과 optimizer 정의

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits = True)

generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)

### Loss Function 정의

In [ ]:
# 네 종류의 loss function 정의
# discriminator_loss: GAN의 판별자 관련 loss function
# generator_loss: GAN의 생성자 관련 loss function
# calc_cycled_loss: 한 이미지와 해당 이미지를 두 번 변환시켜 얻어진 이미지 (말 -> 얼룩말 -> 말 or 얼룩말 -> 말 -> 얼룩말) 사이의 loss를 구하는 loss function
# identity_loss: 한 이미지와 해당 이미지를 동일한 이미지로 유지시키게끔 변환시켜 얻어진 이미지 (말 -> 말 or 얼룩말 -> 얼룩말) 사이의 loss를 구하는 loss function

def discriminator_loss(real, generated):
    ''' Fill here '''


def generator_loss(generated):
    ''' Fill here '''


def calc_cycle_loss(real_image, cycled_image):
    ''' Fill here '''


def identity_loss(real_image, same_image):
    ''' Fill here '''

### 이미지 출력용 함수

In [ ]:
def generate_images(model, test_input):
    prediction = model(test_input)
    
    plt.figure(figsize = (12, 12))

    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']
    
    for i in range(2):
        plt.subplot(1, 2, i + 1)
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

### Training 과정

In [ ]:
def train_step(real_x, real_y):
  
    # Gradient 계산 과정에서 gradient tape가 여러 번 쓰이기 때문에 persistent를 True로 설정함
    with tf.GradientTape(persistent = True) as tape:
        # x: 말 이미지
        # y: 얼룩말 이미지
        # 생성자 g: 변환 x -> y (말 -> 얼룩말)를 담당
        # 생성자 f: 변환 y -> x (얼룩말 -> 말)를 담당
        # fake: 말 -> 얼룩말 or 얼룩말 -> 말
        # cycled: 말 -> 얼룩말 -> 말 or 얼룩말 -> 말 -> 얼룩말
        # same: 말 -> 말 or 얼룩말 -> 얼룩말
    
        fake_y = generator_g(real_x, training = True)
        cycled_x = generator_f(fake_y, training = True)

        fake_x = generator_f(real_y, training = True)
        cycled_y = generator_g(fake_x, training = True)

        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # Generator loss와 cycle loss 계산
        ''' Fill here '''

        # Total generator loss = adversarial loss (generator loss + identity loss) + cycle loss
        ''' Fill here '''

        # Discriminator loss 계산
        ''' Fill here '''
    
    # CycleGAN을 구성하는 생성자와 판별자들의 gradient 값들을 계산하는 과정
    generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)

    # 계산된 gradient 값들을 기반으로 CycleGAN의 생성자와 판별자들을 optimize하는 과정
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))
    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, discriminator_x.trainable_variables))
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, discriminator_y.trainable_variables))

In [ ]:
# 실제 training 과정

for epoch in range(EPOCHS):
    start = time.time()

    n = 0

    for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
        train_step(image_x, image_y)
        if n % 10 == 0:
            print ('.', end = '')
        n += 1

    clear_output(wait = True)
    # 모델의 학습 과정을 확실하게 보여주기 위한 목적으로 동일한 이미지 (sample_horse) 사용
    generate_images(generator_g, sample_horse)

    # Training 과정이 오래 걸리기 때문에, EPOCHS 값을 크게 설정하였을 경우 일정 epoch마다 checkpoint를 생성
    #if (epoch + 1) % 5 == 0:
    #    ckpt_save_path = ckpt_manager.save()
    #    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print ('Time taken for epoch {} is {} sec.\n'.format(epoch + 1, time.time() - start))

## Testing

In [ ]:
# 학습된 모델을 test data에 적용
# 여기서는 말 이미지 중 첫번째 이미지에 적용

for inp in test_horses.take(1):
    generate_images(generator_g, inp)

## Testing 결과 재현

EPOCHS = 50으로 지정한 상태에서 training 과정을 거친 generator g와 f를 불러옴!

In [ ]:
# 저장되어 있는 generator g와 f를 불러옴

generator_g = tf.keras.models.load_model('generator_g.h5')
generator_f = tf.keras.models.load_model('generator_f.h5')

In [ ]:
# 말 -> 얼룩말 Testing 결과 재현

for inp in test_horses.take(1):
    generate_images(generator_g, inp)

In [ ]:
# 얼룩말 -> 말 Testing 결과 재현

for inp in test_zebras.take(1):
    generate_images(generator_f, inp)